In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

## Alabama Page Harvest

In [3]:
pip install openpyxl

### Import Module

In [4]:
# Importing Important libraries
import pandas as pd
from tqdm import tqdm
import re
import time
import urllib
import warnings
from tqdm import trange
import sys
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

### Test

In [70]:
url="https://opencorporates.com/companies/us_ak?action=search_companies&commit=Go&controller=searches&order=&q=google&type=companies&utf8=%E2%9C%93"

In [71]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; apple64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [72]:
res=requests.get(url,headers=headers)

In [73]:
res

<Response [200]>

In [74]:
sou1=BeautifulSoup(res.content,'lxml')

In [75]:
sou1

<html><head><script type="text/javascript"><!--
function leastFactor(n) {
 if (isNaN(n) || !isFinite(n)) return NaN;
 if (typeof phantom !== 'undefined') return 'phantom';
 if (typeof module !== 'undefined' && module.exports) return 'node';
 if (n==0) return 0;
 if (n%1 || n*n<2) return 1;
 if (n%2==0) return 2;
 if (n%3==0) return 3;
 if (n%5==0) return 5;
 var m=Math.sqrt(n);
 for (var i=7;i<=m;i+=30) {
  if (n%i==0)      return i;
  if (n%(i+4)==0)  return i+4;
  if (n%(i+6)==0)  return i+6;
  if (n%(i+10)==0) return i+10;
  if (n%(i+12)==0) return i+12;
  if (n%(i+16)==0) return i+16;
  if (n%(i+22)==0) return i+22;
  if (n%(i+24)==0) return i+24;
 }
 return n;
}
function go() {
 var p=2937285987649; var s=1851820029; var n;
if ((s >> 15) & 1)/*
*13;
*/p+=
32080245*/*
*13;
*/16;/* 120886108*
*/else /*
*13;
*/p-=/* 120886108*
*/12951226* 16;if ((s >> 14) & 1)
p+=
83235624*/*
*13;
*/17;/*
*13;
*/else 	p-=98024776*/*
p+= */15; if ((s >> 13) & 1)
p+=
138471864*/*
p+= */14;/*
else p-=
*

In [52]:
sou.text.strip()

"Business Entity Records | Alabama Secretary of State\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama Secretary of State\n\n\n\n\n\n\n\nMain navigation\n\n\nSOS Office\n\n\nBiography\n\n\nHistory\n\n\nVoter Registration\n\n\nEvent Scheduling\n\n\nFees & Funds\n\n\nFee Schedule\n\n\nFunds Guide\n\n\nVoter List Fee Schedule\n\n\n\n\nCommittees\n\n\nCommittees Guide\n\n\nMeeting Minutes\n\n\n\n\nStudent Center\n\n\nLegislative Internship\n\n\nSOS Internship Application\n\n\nStudent Poll Worker Internship\n\n\nKids Center\n\n\n\n\nQuarterly & Annual Reports\n\n\n\n\nElections\n\n\nServices\n\n\nAdministrative Services\n\n\nAuthentications\n\n\nTrademarks\n\n\nLands\n\n\n\n\nUCC Services\n\n\nUCC Home\n\n\nOnline Filings\n\n\nFiling Fees\n\n\nProcedures\n\n\nUCC Downloads\n\n\n\n\nBusiness Entities\n\n\nOnline Services\n\n\nDomestic Corporations\n\n\nForeign Corporations\n\n\nHomeowner's Associations\n\n\nLLCs\n\n\nLLPs\n\n\nBusiness Downloads\n\n\n\nAthlete Agents\n\n\nOath of Office Information Center\n

In [8]:
cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()

In [9]:
cmp_name

'Frank Dawson Realty Co., Inc.'

### Function

In [10]:
def harvest_page(num):
  loo_df=pd.DataFrame()
  url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'

  headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding":"gzip,deflate,br",
            "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
            "Connection":"keep-alive",
            "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
            "Host":"arc-sos.state.al.us",
            "Sec-Fetch-Dest":"document",
            "Sec-Fetch-Mode":"navigate",
            "Sec-Fetch-Site":"none",
            "Sec-Fetch-User":"?1",
            "Upgrade-Insecure-Requests":"1",
            "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
            "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
            "sec-ch-ua-mobile":"?0",
            "sec-ch-ua-platform":"Windows"
          }

  try:
    res=requests.get(url,headers=headers,timeout=40)
    sou=BeautifulSoup(res.content,'lxml')
    data_find=sou.text.strip()
    try:
        cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
        stus_code=res.status_code
        datas0=[]
        datas1=[]

        for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
          try:
            attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
            detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

            datas1.append(detail_value)
            datas0.append(attri_name)

          except:
            pass

        for i in range(len(datas0)):
          loo_df[datas0[i]]=[str(datas1[i])]

        loo_df['company']=[cmp_name]
        loo_df['Url_number']=[num]
        loo_df['Status']=[stus_code]

        loo_df['Test']=["Allow"]
    except:
       if data_find in "You have exceeded the number accesses allowed.":
        return harvest_page(num)
       else:
           loo_df['Test']=["Not Have"]

    return loo_df
  except:
    return loo_df

In [11]:
#find=harvest_page('000000007')
#harvest_page('000000002')

In [13]:
#find

In [14]:
# dtas_oyut.to_csv('ayeoirtqie.csv')

In [13]:
str_n="000000000"
range_v=[]
for i in range(1,1000):
  st_v=f"{len(str(i))}"
  num_f=str_n[int(st_v):]+str(i)
  range_v.append(num_f)

In [14]:
len(range_v)

999

### Thread Compile


In [ ]:
collect_frame=[]
def compile_run():
    with tqdm(total=len(range_v),desc="harvest") as progress:
        with ThreadPoolExecutor(max_workers=5) as executor:
            for i in executor.map(harvest_page,range_v):
               collect_frame.append(i)
               progress.update()
               time.sleep(.1)
               if len(collect_frame)%5==0:
                time.sleep(.3)
compile_run()

In [ ]:
final_df=pd.DataFrame()
for i in collect_frame:
    final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

In [ ]:
for ciu,i in enumerate(collect_frame):
    print(f'Length {len(i.columns)} --- {ciu}')

In [ ]:
#final_df.to_csv("arc-sos.state Sample 1k out.csv")

In [ ]:
final_df

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Status,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,...,Member Mailing Address,Withdrawn Date,Service of Process,Qualify Date,Filed Date,Place of Formation Changed,Cancelled Date,General Partner Name,General Partner Street Address,General Partner Mailing Address
0,000-000-001,Domestic Corporation,"BIRMINGHAM, AL",Not Provided,200.0,Jefferson County,01/04/1966,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000-000-002,Domestic Corporation,"HUNTSVILLE, AL",Not Provided,200.0,Madison County,04/17/1959,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000-000-003,Domestic Corporation,"MONTGOMERY, AL",Not Provided,200.0,Montgomery County,03/17/1971,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000-000-004,Domestic Non-Profit Corporation,"TALLADEGA, AL",Not Provided,200.0,Talladega County,01/10/1985,"JENKINS, ANN","206 TINNEY STREETTALLADEGA, AL 35160",Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000-000-005,Domestic Corporation,"AUBURN, AL",Not Provided,200.0,Lee County,01/01/1969,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,000-000-202,Domestic Corporation,Not Provided,Not Provided,200.0,Montgomery County,01/19/2011,BUSINESS FILINGS INCORPORATED,"2 N JACKSON ST STE 605MONTGOMERY, AL 36104","2 N JACKSON ST STE 605MONTGOMERY, AL 36104",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,000-000-203,Domestic Corporation,"3636 MENLO COURTSMONTGOMERY, AL 36116","3636 MENLO COURTSMONTGOMERY, AL 36116",200.0,Montgomery County,01/20/2011,"THOMAS, LAYONS","3636 MENLO COURTSMONTGOMERY, AL 36116","3636 MENLO COURTSMONTGOMERY, AL 36116",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,000-000-204,Domestic Corporation,Not Provided,Not Provided,200.0,Shelby County,01/13/2011,"BUECHLER, CHRISTOPHER J","430 NORTH LAKE RDBIRMINGHAM, AL 35242","430 NORTH LAKE RDBIRMINGHAM, AL 35242",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,000-000-205,Domestic Corporation,"BIRMINGHAM, AL",Not Provided,200.0,Jefferson County,07/17/1972,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
datas=pd.DataFrame()

In [ ]:
datas['work']=[10,10,10,10,10]
datas['Do']=[4,5,6,2,8]

In [ ]:
datas

,work,Do
0,10,4
1,10,5
2,10,6
3,10,2
4,10,8


### Proxci And User Agent

In [5]:
procxi_df=pd.read_csv('/content/Free_Proxy_List.csv')
user_agent_df=pd.read_csv("/content/100 User Agent Pass Frame.csv")

In [6]:
procxi_list=procxi_df['ip'].to_list()
user_agent_list=user_agent_df['User_Agents'].to_list()

### Async Function

In [7]:
### Async Code

import aiohttp
import asyncio
import nest_asyncio

In [8]:
import random

In [9]:
user_agent_list_len=range((len(user_agent_list)))
user_agent_random_num=random.choice(user_agent_list_len)
print(user_agent_list[user_agent_random_num])

Motorola Moto G Power (2021)


In [39]:
async def harvest_page_asy(session,num):
        loo_df=pd.DataFrame()
        try:
          headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,application/signed-exchange;v=b3;q=0.7",
                        "Accept-Encoding":"gzip,deflate,br",
                        "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
                        "Connection":"keep-alive",
                        "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
                        "Host":"arc-sos.state.al.us",
                        "Sec-Fetch-Dest":"document",
                        "Sec-Fetch-Mode":"navigate",
                        "Sec-Fetch-Site":"none",
                        "Sec-Fetch-User":"?1",
                        "Upgrade-Insecure-Requests":"1",
                        "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
                        "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
                        "sec-ch-ua-mobile":"?0",
                        "sec-ch-ua-platform":"Windows"
                      }
          url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'
          async with session.get(url, headers=headers,timeout=10) as response:
              status_code = response.status
              page_content = await response.text()
              #res=requests.get(url,headers=headers,timeout=40)
              sou=BeautifulSoup(page_content,'lxml')
              data_find=sou.text.strip()
              try:
                  cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
                  stus_code=res.status_code
                  datas0=[]
                  datas1=[]

                  for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
                    try:
                      attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
                      detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

                      datas1.append(detail_value)
                      datas0.append(attri_name)

                    except:
                      pass

                  for i in range(len(datas0)):
                    loo_df[datas0[i]]=[str(datas1[i])]

                  loo_df['company']=[cmp_name]
                  loo_df['Url_number']=[num]
                  loo_df['Status']=[stus_code]

                  loo_df['Test']=["Allow"]
              except:
                if data_find in "You have exceeded the number accesses allowed.":
                  time.sleep(1.5)
                  return harvest_page(num)
                else:
                  loo_df['Url_number']=[num]
                  loo_df['Test']=["Not Have"]

              return loo_df
        except:
          loo_df['Url_number']=[num]
          loo_df['Test']=["Error"]
          time.sleep(.3)
          return loo_df


#### Async Compile

In [ ]:
collect_frame=[]
async def main(list_num):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for perma in list_num:
            tasks.append(harvest_page(session, perma))
        results = []
        with tqdm(total=len(list_num),desc="harvest") as progress:
          for task in asyncio.as_completed(tasks):
              data=await task
              collect_frame.append(data)
              time.sleep(.2)
              progress.update(1)
              if len(collect_frame)%6==0:
                time.sleep(1)


if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main(range_v[:1000]))

In [140]:
final_df=pd.DataFrame()
for i in collect_frame:
    final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

In [142]:
final_df

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Status,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,...,Transaction Date,Legal Name Changed From,company,Url_number,Test,Registered Agent Changed From,Document Date / Type / Pages,Dissolved Date,Legal Name Merged,Miscellaneous Filing Entry
0,000-000-003,Domestic Corporation,"MONTGOMERY, AL",Not Provided,200.0,Montgomery County,03/17/1971,Not Provided,Not Provided,Not Provided,...,11/10/1977,A. Fence City of Alabama,"A-1 Hurricane Fence Industries of Montgomery, ...",000000003,Allow,NaN,NaN,NaN,NaN,NaN
1,000-000-005,Domestic Corporation,"AUBURN, AL",Not Provided,200.0,Lee County,01/01/1969,Not Provided,Not Provided,Not Provided,...,NaN,NaN,"A-O Rents, Inc.",000000005,Allow,NaN,NaN,NaN,NaN,NaN
2,000-000-004,Domestic Non-Profit Corporation,"TALLADEGA, AL",Not Provided,200.0,Talladega County,01/10/1985,"JENKINS, ANN","206 TINNEY STREETTALLADEGA, AL 35160",Not Provided,...,01/25/1985,NaN,East Alabama Children's Trust Fund Council,000000004,Allow,"JENKINS, ANNPOST OFFICE BOX 367TALLADEGA, AL 3...",01/25/1985 Registered Agent Change 2 pgs.,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,000000006,Not Have,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,000000007,Not Have,NaN,NaN,NaN,NaN,NaN
5,000-000-008,Domestic Corporation,"FLORENCE, AL",Not Provided,200.0,Lauderdale County,09/15/1969,Not Provided,Not Provided,Not Provided,...,NaN,NaN,"A-One Body Shop of Florence, Inc.",000000008,Allow,NaN,05/26/1995 Dissolution 2 pgs.,05/26/1995,NaN,NaN
6,000-000-010,Domestic Corporation,"LEEDS, AL",Not Provided,200.0,Saint Clair County,01/15/1970,Not Provided,Not Provided,Not Provided,...,NaN,NaN,"A-1 Garbage Service, Inc.",000000010,Allow,NaN,NaN,NaN,NaN,NaN
7,000-000-002,Domestic Corporation,"HUNTSVILLE, AL",Not Provided,200.0,Madison County,04/17/1959,Not Provided,Not Provided,Not Provided,...,12/05/1985,Rental House Inc.,"Frank Dawson Realty Co., Inc.",000000002,Allow,NaN,10/03/1986 Dissolution 7 pgs.,10/03/1986,"AAA Equipment Co., Inc.",AMENDMENT FILED
8,000-000-009,Domestic Corporation,"SELMA, AL",Not Provided,200.0,Dallas County,03/14/1972,Not Provided,Not Provided,Not Provided,...,NaN,NaN,"A-1 Furniture Co., Inc.",000000009,Allow,NaN,09/23/2011 Dissolution 3 pgs.,09/21/2011,NaN,NaN
9,000-000-001,Domestic Corporation,"BIRMINGHAM, AL",Not Provided,200.0,Jefferson County,01/04/1966,Not Provided,Not Provided,Not Provided,...,09/14/1970,"H & M Bonding Company, Inc.","A Bonding, Inc.",000000001,Allow,NaN,NaN,NaN,NaN,NaN


In [90]:
final_df.groupby('Test').count()

,Url_number
Test,
Error,100


In [91]:
final_df

,Url_number,Test
0,000000094,Error
1,000000021,Error
2,000000095,Error
3,000000022,Error
4,000000096,Error
...,...,...
95,000000089,Error
96,000000090,Error
97,000000091,Error
98,000000092,Error


In [ ]:
final_df.sort_values(['Url_number'], ascending=[True])

In [ ]:
final_df[final_df['Test']=="Error"]['Url_number'].to_list()

### Loop Complies

In [43]:
tests_range=range_v[:10]

In [44]:
find_final_data=pd.DataFrame()

In [45]:
while True:
    collect_frame=[]
    async def main(list_num):
        tasks = []
        async with aiohttp.ClientSession() as session:
            for perma in list_num:
                tasks.append(harvest_page_asy(session, perma))
            results = []
            with tqdm(total=len(list_num),desc="harvest") as progress:
                for task in asyncio.as_completed(tasks):
                    data=await task
                    time.sleep(.3)
                    collect_frame.append(data)
                    progress.update(1)
                    if len(collect_frame)%10==0:
                        time.sleep(2)


    if __name__ == "__main__":
        nest_asyncio.apply()
        asyncio.run(main(tests_range))

    final_df=pd.DataFrame()
    for i in collect_frame:
        final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

    find_df=final_df[final_df['Test']!="Error"]
    tests_range=final_df[final_df['Test']=="Error"]['Url_number'].to_list()

    find_final_data=pd.concat([find_df,find_final_data],axis=0)

    print(f"Total Count : {len(collect_frame)}")
    print(f"Find Count : {len(find_df)}")
    print(f"Not Fount : {len(tests_range)}")

    if len(tests_range)==0:
        print('Break')
        break


harvest: 100%|██████████| 10/10 [00:41<00:00,  4.12s/it]


Total Count : 10
Find Count : 1
Not Fount : 9


harvest: 100%|██████████| 9/9 [00:05<00:00,  1.66it/s]


Total Count : 9
Find Count : 4
Not Fount : 5


harvest: 100%|██████████| 5/5 [00:03<00:00,  1.52it/s]


...

Total Count : 1
Find Count : 0
Not Fount : 1


harvest: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it]


Total Count : 1
Find Count : 0
Not Fount : 1


harvest:   0%|          | 0/1 [00:01<?, ?it/s]


KeyboardInterrupt: ignored

In [41]:
tests=find_final_data[['Url_number','Test']]

In [42]:
find_final_data

,Url_number,Test
1,000000008,Not Have
2,000000009,Not Have
3,000000005,Not Have
4,000000001,Not Have
2,000000004,Not Have
3,000000006,Not Have
4,000000007,Not Have
5,000000002,Not Have
7,000000003,Not Have
